This gets a unix timestamp input, converts it to a pandas compatible datetime, runs it against known hourly data, and returns a temperature. Linear interpolation will be used between two hourly data points. 

Maybe some optimization could be done if the two hourly temps have changed less than a certain amount.

In [2]:
from netCDF4 import Dataset, MFDataset, num2date
import pandas as pd
import numpy as np
from zipfile import ZipFile
import matplotlib.pyplot as plt
import matplotlib as ply
from datetime import datetime, timedelta

at_data = Dataset("../data/air_temperature.nc")
air_temperature = at_data.variables['air_temperature'][:]
time = at_data.variables['time'][:]

In [5]:
time_var = at_data.variables['time']
dates = num2date(time[:], time_var.units)
dates_pd = pd.to_datetime(dates)

In [6]:
df = pd.DataFrame(dates_pd,columns=['Date'])
df['air_temperature'] = air_temperature[:]
df = df.set_index('Date')

In [86]:
pumaRange = df.loc['2018-10':'2019-05']
type(pumaRange.index)

pandas.core.indexes.datetimes.DatetimeIndex

The way the algorithm will work is that it will check the timestamp that is input and see if it is +/- 15 minutes from the top of the hour. If it is, then the returned temperature will just be that hourly datapoint. If it is not, then two weather points before and after the input time, then the returned temperature will be an interpolation between the two

In [111]:
def outdoorTemp(inUnix):
    pumaPoint = datetime.utcfromtimestamp(inUnix)
    nearestIndex = pumaRange.index.get_loc(pumaPoint,method='nearest')
    
    print(nearestIndex)
    nearestTemp = pumaRange.iloc[nearestIndex]
    print(type(nearestTemp.name))

    print(type(abs(pumaPoint - nearestTemp.name)))
    #print(pumaRange[2232])
    
outdoorTemp(1546386804) #2019-1-1 23:53:24

2232
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
